In [7]:
! pip install torch

In [22]:
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

<h2>予測コード</h2>


In [23]:
import torch
import torch.nn as nn

class SimpleGPTPredictor(nn.Module):
    def __init__(self, vocab_size, embed_size, num_heads):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)

        # K, V, Qとかの計算->残渣結合->正規化->FFN (Feed Forward Network)とかをやってるぽい
        # Attention計算では、QとKの（各トークンのKにたいする）内積を計算して、それをベクトルとしてもつ。そのベクトルをSoftmaxで重みにする。
        # 各トークンのVと重みつき平均を取る。
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(embed_size, num_heads),
            num_layers=2
        )

        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(embed_size, num_heads),
            num_layers=2
        )
        
        # Linearは、行列積 shapeは？
        # 
        self.lm_head = nn.Linear(embed_size, vocab_size)

    def forward(self, src, tgt):
        # ソースをエンコード
        src_embedded = self.embedding(src)
        encoded = self.encoder(src_embedded)
        
        # ターゲットをデコード
        tgt_embedded = self.embedding(tgt)
        
        # ★追加3: 因果マスク
        tgt_mask = self.generate_square_subsequent_mask(tgt.size(0))
        
        decoded = self.decoder(tgt_embedded, encoded, tgt_mask=tgt_mask)
        output = self.lm_head(decoded)
        
        return output
        
    # ★追加4: マスク生成メソッド
    def generate_square_subsequent_mask(self, sz):
        mask = torch.triu(torch.ones(sz, sz, device=device)) == 1
        mask = mask.transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

with open('inputLearnText.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = list(set(text))  #hello->heloみたいにする。
char_to_id = {ch: i for i, ch in enumerate(chars)} # 文字: idの配列
id_to_char = {i: ch for i, ch in enumerate(chars)} # id: 文字の配列

print("文字→ID辞書:", char_to_id)

# テキストを1文字ずつIDに変換
def text_to_ids(text):
    return [char_to_id[ch] for ch in text]

# IDを1文字ずつテキストに変換
def ids_to_text(ids):
    return ''.join([id_to_char[i] for i in ids])



文字→ID辞書: {'-': 0, 'æ': 1, ' ': 2, 'έ': 3, 'm': 4, 'X': 5, 'α': 6, 'G': 7, 'ה': 8, 'ἐ': 9, 'R': 10, 'D': 11, 'V': 12, '[': 13, 'כ': 14, ':': 15, 'ῳ': 16, 'a': 17, 'c': 18, 'q': 19, 'p': 20, '6': 21, 'ν': 22, 'w': 23, 'ω': 24, 'ᾶ': 25, 'ή': 26, '”': 27, 'E': 28, 'L': 29, 'O': 30, '8': 31, '.': 32, '3': 33, 'e': 34, 'f': 35, 'U': 36, 'b': 37, 'j': 38, '5': 39, 'ώ': 40, 'מ': 41, 'ῃ': 42, 'Z': 43, '9': 44, '“': 45, 'ἀ': 46, 'P': 47, 'y': 48, 'τ': 49, 'σ': 50, 'μ': 51, 'W': 52, 'ρ': 53, '4': 54, 'i': 55, 'ο': 56, 'v': 57, 'ῆ': 58, 't': 59, 'ς': 60, 'ε': 61, 'κ': 62, 'A': 63, 'λ': 64, 'ὕ': 65, 'ָ': 66, 'n': 67, 'k': 68, 'י': 69, 'T': 70, 'M': 71, 'o': 72, 'ῥ': 73, 'υ': 74, 'x': 75, 'ἰ': 76, 'l': 77, 'Æ': 78, ';': 79, '(': 80, 'F': 81, 'u': 82, 'ό': 83, ']': 84, '\n': 85, 'ι': 86, 'ו': 87, 'C': 88, '?': 89, 'χ': 90, 'I': 91, ',': 92, 'Y': 93, 'z': 94, '2': 95, 'H': 96, '1': 97, 'ō': 98, 'η': 99, 'B': 100, 'ְ': 101, 'N': 102, 'ύ': 103, 'ί': 104, '0': 105, ')': 106, 'd': 107, 'π': 108, 's': 109,

In [11]:
model = SimpleGPTPredictor(vocab_size=len(chars), embed_size=32, num_heads=4)


<h2>学習用コード</h2>

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim

# テキストを1文字ずつIDに変換
def text_to_ids(text):
    return [char_to_id[ch] for ch in text]

# IDを1文字ずつテキストに変換
def ids_to_text(ids):
    return ''.join([id_to_char[i] for i in ids])

# 簡単な学習データ作成
def create_training_data(text, seq_len=10):
    ids = text_to_ids(text)
    src_data, tgt_data = [], []

    for i in range(len(ids) - seq_len):
        src_data.append(ids[i:i+seq_len])      # 入力：10文字
        tgt_data.append(ids[i+1:i+seq_len+1])  # 正解：1文字ずらした10文字

    return torch.tensor(src_data, device=device), torch.tensor(tgt_data, device=device)


# 学習データ作成
# 追加する
with open('inputLearnText.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# これなんだっけ
chars = set(text)

char_to_id = {ch: i for i, ch in enumerate(chars)} # 文字: idの配列
id_to_char = {i: ch for i, ch in enumerate(chars)} # id: 文字の配列

train_inputs, train_targets = create_training_data(text)

print(f"学習データ数: {len(train_inputs)}")
print(f"例 - 入力: '{ids_to_text(train_inputs[20].tolist())}'")
print(f"例 - 正解: '{ids_to_text(train_targets[20].tolist())}'")


model = SimpleGPTPredictor(vocab_size=len(chars), embed_size=32, num_heads=4)
model.to(device)
# 学習設定
optimizer = optim.Adam(model.parameters(), lr=0.001)
# 損失関数の種類
criterion = nn.CrossEntropyLoss()

print("\n学習開始...")
# range
train_inputs = train_inputs[:5000]
train_targets = train_targets[:5000]

train_src, train_tgt = create_training_data(text)

for epoch in range(10):
    total_loss = 0
    batch_size = 256
    
    for i in range(0, len(train_src), batch_size):
        optimizer.zero_grad()
        
        src_batch = train_src[i:i+batch_size]
        tgt_batch = train_tgt[i:i+batch_size]
        
        # ★変更: 2つの引数を渡す
        output = model(src_batch, tgt_batch)
        
         # (batch, seq, vocab) ->  #(seq, )
        # 
        # ------- before -------
        # 
        # 
        # original_tensor = [
        # [  # バッチ1
        #     [1, 2, 3, 4],  # 位置1の予測ベクトル
        #     [5, 6, 7, 8],  # 位置2の予測ベクトル  
        #     [9,10,11,12]   # 位置3の予測ベクトル
        # ],
        # [  # バッチ2
        #     [13,14,15,16], # 位置1の予測ベクトル
        #     [17,18,19,20], # 位置2の予測ベクトル
        #     [21,22,23,24]  # 位置3の予測ベクトル
        # ]
        # ]
        #
        # ------- after -------
        #         reshaped_tensor = [
        #   [1, 2, 3, 4],   # バッチ1-位置1
        #   [5, 6, 7, 8],   # バッチ1-位置2  
        #   [9,10,11,12],   # バッチ1-位置3
        #   [13,14,15,16],  # バッチ2-位置1
        #   [17,18,19,20],  # バッチ2-位置2
        #   [21,22,23,24]   # バッチ2-位置3
        # ]
        # train_targetsは(batch*seq)の一次元。要するに次元を減らしている。（数値の変更とかはない）
        loss = criterion(output.view(-1, len(chars)), tgt_batch.view(-1))
        loss.backward()

        
        optimizer.step()
        
        total_loss += loss.item()
    
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss/len(train_src):.4f}")


学習データ数: 51642
例 - 入力: ' Fathers w'
例 - 正解: 'Fathers wi'

学習開始...


/opt/anaconda3/lib/python3.13/site-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 0, Loss: 0.0031


KeyboardInterrupt: 

<h2>予測テスト</h2>

In [ ]:
def test_prediction(model: SimpleGPTPredictor, input_text):
    input_ids = text_to_ids(input_text)
    input_tensor = torch.tensor([input_ids])

    with torch.no_grad():
        output = model(input_tensor)
        last_char_probs = output[0, -1, :]
        probs = torch.softmax(last_char_probs, dim=-1)

        # 一位のトークンを呼び出す。
        top_prob, top_index = torch.topk(probs, 1)
        char_id = top_index.item()  # テンソルから数値を取り出し
        predicted_char = id_to_char[char_id]  # IDを文字に変換

        return predicted_char

def generateSeq(model, text, count = 0):
    nextSingleToken = test_prediction(model, text)
    if(count < 20):
        return generateSeq(model, text+nextSingleToken, count+1)
    else:
        return text+nextSingleToken

prompt = "The god is a"
completion = generateSeq(model, prompt)
print("入力テキスト: ", prompt)
print("回答: ", completion)

入力テキスト:  The god is a
回答:  The god is aS7ggugÆGW)vπηrb“7gugu
